TODO:
- [ ] better way to visualise detections, maybe [imgviz](https://github.com/wkentaro/imgviz)

# Torch

Heavily based on the [TorchVision Object Detection Finetuning Tutorial](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html).

In [1]:
%%bash

pip install cython

# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-8wy9ki7p
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-8wy9ki7p


## Step 1. Download and define the dataset

Download the [MineApple fruit detection dataset](https://rsn.umn.edu/projects/orchard-monitoring/minneapple). This should take ~30 seconds.

In [2]:
%%bash

wget --no-clobber https://conservancy.umn.edu/bitstream/handle/11299/206575/detection.tar.gz -O data/detection.tar.gz
tar -xvf data/detection.tar.gz --directory data/

detection/test/images/dataset2_back_630.png
detection/train/masks/20150919_174151_image11.png
detection/train/images/20150919_174730_image161.png
detection/train/masks/20150921_131346_image1121.png
detection/train/masks/20150921_131234_image546.png
detection/test/images/dataset3_front_510.png
detection/train/images/20150921_131346_image1121.png
detection/train/images/
detection/train/masks/20150921_132245_image396.png
detection/train/masks/20150921_131453_image676.png
detection/train/masks/20150921_131833_image296.png
detection/train/images/20150919_174151_image301.png
detection/train/images/20150921_132038_image76.png
detection/train/masks/20150921_131453_image1196.png
detection/train/images/20150921_131346_image51.png
detection/train/masks/20150921_132038_image1241.png
detection/test/images/dataset4_front_900.png
detection/test/images/dataset4_front_1020.png
detection/train/images/20150919_174730_image216.png
detection/train/images/20150921_131453_image1196.png
detection/train/images

File ‘data/detection.tar.gz’ already there; not retrieving.


In [3]:
from pathlib import Path

import numpy as np
import torch
import torch.utils.data
from PIL import Image


class MinneAppleDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = Path(root)
        self.transforms = transforms

        self.image_dir = root / "images"
        self.mask_dir = root / "masks"
        self.image_paths = list(sorted(self.image_dir.glob("*.png")))
        self.mask_paths = [self.mask_dir / x.name for x in self.image_paths]
        assert all(x.exists() for x in self.mask_paths), "Missing mask files."

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        mask_path = self.mask_paths[index]

        image = Image.open(image_path).convert("RGB")
        mask = np.array(Image.open(mask_path))

        object_ids = np.unique(mask)
        object_ids = object_ids[1:]
        masks = mask == object_ids[:, None, None]

        boxes = []
        for object_mask in masks:
            pixel_indices = np.where(object_mask)
            xmin = np.min(pixel_indices[1])
            xmax = np.max(pixel_indices[1])
            ymin = np.min(pixel_indices[0])
            ymax = np.max(pixel_indices[0])
            # TODO: Why are there bad bboxes?
            if xmin == xmax or ymin == ymax:
                continue
            boxes.append([xmin, ymin, xmax, ymax])

        num_objects = len(object_ids)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objects,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([index])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objects,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            image, target = self.transforms(image, target)

        return image, target

    def __len__(self):
        return len(self.image_paths)

## Step 2. Train a fine-tuned model

In [4]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_instance_segmentation_model(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    return model

In [ ]:
%%bash

git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [ ]:
from engine import train_one_epoch, evaluate
import transforms as T
import utils


def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
root = Path("data/detection/")
train_root = root / "train"

In [ ]:
dataset = MinneAppleDataset(train_root, get_transform(train=True))
dataset_test = MinneAppleDataset(train_root, get_transform(train=False))

torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2
model = get_instance_segmentation_model(num_classes)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
from torch.optim.lr_scheduler import StepLR
num_epochs = 1

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

## Step 3. Inference

In [ ]:
img, _ = dataset_test[0]
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

In [ ]:
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())